In [3]:
import xgboost
import shap
import matplotlib.pylab as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import roc_auc_score, average_precision_score
import argparse
from sklearn.model_selection import GridSearchCV
import pickle
from tqdm import tqdm

In [4]:
def bootstrap_ci(y_pre, y_label, sample_size, repetitions = 1000, alpha = 0.05): 
    y_pre = np.array(y_pre)
    y_label = np.array(y_label)
    
    auc = []
    ap = []
    for i in range(repetitions):
        np.random.seed(i)
        idx = list(np.random.choice(len(y_pre), replace = True, size = sample_size))
        y_pre_bootstrap = y_pre[idx]
        y_label_bootstrap = y_label[idx]
        auc.append(roc_auc_score(y_label_bootstrap, y_pre_bootstrap))
        ap.append(average_precision_score(y_label_bootstrap, y_pre_bootstrap))
    # confidence interval
    left_auc = np.percentile(auc, alpha/2*100)
    right_auc = np.percentile(auc, 100-alpha/2*100)
    left_ap = np.percentile(ap, alpha/2*100)
    right_ap = np.percentile(ap, 100-alpha/2*100)
    # point estimate
    print('average AUROC', np.mean(auc))
    print((1-alpha)*100,'%','confidence interval for the AUROC:', (round(left_auc,4), round(right_auc,4)))
    print('average AP', np.mean(ap))
    print((1-alpha)*100,'%','confidence interval for the AP:', (round(left_ap,4), round(right_ap,4)))
    return auc, left_auc, right_auc, ap, left_ap, right_ap

In [5]:
path = '/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/reelim14/'
if not os.path.isdir(path):
    os.mkdir(path)

In [6]:
X = pd.read_excel('/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/reelim14.xlsx')
y = X['30_days_label']

drop_list = ['30_days_label']
X = X.drop(drop_list, axis=1)

fea_list = pd.read_excel('/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/feature_list_reelim.xlsx')
nominal_fea = fea_list[fea_list['Nominal']==1]['Type_Short_Name'].tolist()
nominal_fea = list(set(nominal_fea) & set(X.columns))
X = pd.get_dummies(X, columns=nominal_fea, drop_first=True)

display_name = pd.read_excel('/Users/kj/Library/CloudStorage/OneDrive-한림대학교/연구 및 과제/박사논문/feature_list_Display_name_reelim.xlsx')
display_col=[]
for col in X.columns:
    matching_display_names = display_name.loc[display_name['Type_Short_Name'] == col, 'Display_Name']
    if not matching_display_names.empty:
        display_col.append(matching_display_names.iloc[0])
    else:
        display_col.append(col)  # 일치하는 Display_Name이 없는 경우 원래의 컬럼 이름 사용

col_dict = dict(zip(X.columns, display_col))

print(X.columns)
print('After encoding', X.shape)

Index(['MIC: ETP', 'MIC: IMP or MPM', 'ICU days_before rectal swab (+)',
       'Admission day_before rectal swab (+)', 'Age', 'BMI', 'WBC, Low',
       'Neutrophil count, Low', 'RBC', 'Hb', 'RDW', 'PLT, High', 'MPV', 'PT',
       'aPTT', 'Albumin', 'ALP', 'AST', 'ALT', 'BUN', 'Cr',
       'total calcium, High', 'Total Cheloesterol', 'GGT', 'Direct  bilirubin',
       'Total bilirubin', 'LD', 'Na, Low', 'K, Low', 'cl, Low', 'uric acid',
       'tCO2, High', 'protein', 'CRP', 'No. additional colonization sites',
       'No. other CRGNB in any sites', 'other blood pathogen_CRAB_1', 'CPE_1',
       'other blood pathogen_candida_1', 'other blood pathogen_VRE_1',
       'other blood pathogen_CRPA_1', 'Rectal spp._Enterobacter spp._1',
       'Rectal spp._Other klebsiella_1', 'Other blood pathogen_bacteria_1',
       'ABO_A_1', 'Carbapenemase_NDM_1', 'Rectal spp._KPN_1',
       'Rectal spp._S. marcescens_1', 'colonization_Urine_1',
       'colonization_BAL_1', 'Rectal spp._Others_1', 'Rectal

In [7]:
print(X.columns)
print(X.shape)
print('# samples: ', X.shape[0])
print('# positive samples: ', sum(y==1))
print('# negative samples: ', sum(y==0))
print('# features: ', X.shape[1])  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=7)

y_train = np.array(y_train); y_test = np.array(y_test); y_val = np.array(y_val)

Index(['MIC: ETP', 'MIC: IMP or MPM', 'ICU days_before rectal swab (+)',
       'Admission day_before rectal swab (+)', 'Age', 'BMI', 'WBC, Low',
       'Neutrophil count, Low', 'RBC', 'Hb', 'RDW', 'PLT, High', 'MPV', 'PT',
       'aPTT', 'Albumin', 'ALP', 'AST', 'ALT', 'BUN', 'Cr',
       'total calcium, High', 'Total Cheloesterol', 'GGT', 'Direct  bilirubin',
       'Total bilirubin', 'LD', 'Na, Low', 'K, Low', 'cl, Low', 'uric acid',
       'tCO2, High', 'protein', 'CRP', 'No. additional colonization sites',
       'No. other CRGNB in any sites', 'other blood pathogen_CRAB_1', 'CPE_1',
       'other blood pathogen_candida_1', 'other blood pathogen_VRE_1',
       'other blood pathogen_CRPA_1', 'Rectal spp._Enterobacter spp._1',
       'Rectal spp._Other klebsiella_1', 'Other blood pathogen_bacteria_1',
       'ABO_A_1', 'Carbapenemase_NDM_1', 'Rectal spp._KPN_1',
       'Rectal spp._S. marcescens_1', 'colonization_Urine_1',
       'colonization_BAL_1', 'Rectal spp._Others_1', 'Rectal

In [8]:
feature_num_list = [X_train.shape[1], 151, 145, 140, 135, 130, 125, 120, 115, 110, 105, 100, 95, 90, 87, 80, 75, 70, 64, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 14, 13, 12, 11, 10,9,8,7,6,5,4,3,2,1]
X_train_all = X_train.copy()
X_test_all = X_test.copy()
X_val_all = X_val.copy()

features_ranking_dict = {}
auc_dict = {}
ap_dict = {}
left_auc_dict = {}
right_auc_dict = {}
left_ap_dict = {}
right_ap_dict = {}
ranked_features = X_train.columns

In [9]:
features_input = {}
ranked_features = X_train_all.columns
for feature_num in feature_num_list:
    print('# features: ', feature_num)
    X_train = X_train_all.loc[:, ranked_features[:feature_num]]
    X_test = X_test_all.loc[:, ranked_features[:feature_num]]
    X_val = X_val_all.loc[:, ranked_features[:feature_num]]
    features_input[feature_num] = ranked_features[:feature_num]
    
    # early_stopping_rounds를 생성자에 포함
    if feature_num > 2:
        xlf = xgboost.XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=3, subsample=0.8, min_child_weight=1, 
                                    objective='binary:logistic', random_state=7, early_stopping_rounds=100)
    else:
        xlf = xgboost.XGBClassifier(n_estimators=1000, subsample=0.5, 
                                    objective='binary:logistic', random_state=7, early_stopping_rounds=100)
    
    # fit 메서드에서 early_stopping_rounds 매개변수를 제거
    xlf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    model_train = xlf
    pickle.dump(model_train, open(path + "model_" + str(feature_num) + ".pickle.dat", "wb"))
    
    y_pre = model_train.predict_proba(X_test)[:, 1]
    auc, left_auc, right_auc, ap, left_ap, right_ap = bootstrap_ci(y_pre, y_test, len(y_test), repetitions=1000, alpha=0.05)
    auc_dict[feature_num] = auc
    ap_dict[feature_num] = ap
    left_auc_dict[feature_num] = left_auc
    right_auc_dict[feature_num] = right_auc
    left_ap_dict[feature_num] = left_ap
    right_ap_dict[feature_num] = right_ap    
    
    if len(X_train) >= 5000:
        back_data = X_train.sample(n=5000, random_state=428)
    else:
        back_data = X_train
    
    if len(X_test) >= 2000:
        fore_data = X_test.sample(n=2000, random_state=528)
        fore_data_label = y_test.sample(n=2000, random_state=528)  # y_test를 직접 샘플링
    else:
        fore_data = X_test
        fore_data_label = y_test  # 전체 y_test를 사용

X_test = X_test.astype({col: int for col in X_test.select_dtypes(include=bool).columns})
X_train = X_train.astype({col: int for col in X_train.select_dtypes(include=bool).columns})

explainer = shap.TreeExplainer(model_train, data=back_data)
shap_values = explainer.shap_values(fore_data, check_additivity=False)
ranked_features = X_train.columns[np.argsort(-np.sum(np.abs(shap_values), axis=0))]
features_ranking_dict[feature_num] = ranked_features


# features:  64
average AUROC 0.8211300921418818
95.0 % confidence interval for the AUROC: (0.6106, 0.954)
average AP 0.41609063634087523
95.0 % confidence interval for the AP: (0.133, 0.6966)
# features:  151
average AUROC 0.8211300921418818
95.0 % confidence interval for the AUROC: (0.6106, 0.954)
average AP 0.41609063634087523
95.0 % confidence interval for the AP: (0.133, 0.6966)
# features:  145
average AUROC 0.8211300921418818
95.0 % confidence interval for the AUROC: (0.6106, 0.954)
average AP 0.41609063634087523
95.0 % confidence interval for the AP: (0.133, 0.6966)
# features:  140
average AUROC 0.8211300921418818
95.0 % confidence interval for the AUROC: (0.6106, 0.954)
average AP 0.41609063634087523
95.0 % confidence interval for the AP: (0.133, 0.6966)
# features:  135
average AUROC 0.8211300921418818
95.0 % confidence interval for the AUROC: (0.6106, 0.954)
average AP 0.41609063634087523
95.0 % confidence interval for the AP: (0.133, 0.6966)
# features:  130
average AUROC 

In [8]:
pickle.dump(auc_dict, open(path+"auc_dict.pickle.dat", "wb"))
pickle.dump(ap_dict, open(path+"ap_dict.pickle.dat", "wb"))
pickle.dump(left_auc_dict, open(path+"left_auc_dict.pickle.dat", "wb"))
pickle.dump(right_auc_dict, open(path+"right_auc_dict.pickle.dat", "wb"))
pickle.dump(left_ap_dict, open(path+"left_ap_dict.pickle.dat", "wb"))
pickle.dump(right_ap_dict, open(path+"right_ap_dict.pickle.dat", "wb"))
pickle.dump(features_ranking_dict, open(path+"features_ranking_dict.pickle.dat", "wb"))
pickle.dump(features_input, open(path+"features_input.pickle.dat", "wb"))

In [9]:
with open(path+'features_ranking_dict.pickle.dat', 'rb') as f:
    features_ranking_dict = pickle.load(f)

In [10]:
# features_ranking_dict를 데이터프레임으로 변환합니다.
df = pd.DataFrame(list(features_ranking_dict.items()), columns=['Feature', 'Ranking'])

# 데이터프레임을 엑셀 파일로 내보냅니다.
df.to_excel('features_ranking.xlsx', index=False)